In [4]:
from starlette.applications import Starlette
from starlette.websockets import WebSocketDisconnect
import uvicorn, asyncio, json
import nest_asyncio
nest_asyncio.apply()

In [13]:
state = {'web_client': False, 'python_client': False}
websocket_dict = {}

app = Starlette()

async def close_client(websocket, msg):
    await websocket.send_json({'event': 'close', 'msg': msg})
    await websocket.close()

async def send_python_status(websocket=None, status=None):
    if websocket is None:
        if state['web_client']: websocket = websocket_dict['web_client']
        else: return
    if status is None: status = 'Online' if state['python_client'] else 'Offline'
    await websocket.send_json({'event': 'pythonClientStatus', 'msg': status})

async def handle_web_client(websocket):
    while True:
        try:
            await send_python_status(websocket)
            msg = await websocket.receive_json()
#             if msg['event'] == 'getPythonClientStatus':
#                 await send_python_status(websocket)
#             else:
#                 print(f'Invalid event: ({msg["event"]!r}).')
        except json.JSONDecodeError: print('Invalid JSON')
        except WebSocketDisconnect: break
        
async def handle_python_client(websocket):
    await send_python_status()
    while True:
        try:
            msg = await websocket.receive_json()
            await websocket_dict['web_client'].send_json(msg)
        except json.JSONDecodeError: print('Invalid JSON')
        except WebSocketDisconnect:
            await send_python_status(status='Offline')
            break

func_dict = {'web_client': handle_web_client, 'python_client': handle_python_client}

@app.websocket_route('/ws')
async def websocket_endpoint(websocket):
    await websocket.accept()
    msg = await websocket.receive_json()
    client_type = msg['client']
    if client_type not in state:
        await close_client(websocket, f'Invalid client_type ({client_type!r})')
        return
    if state[client_type]:
        await close_client(websocket, 'Client already connected')
        return
    
    state[client_type] = True
    websocket_dict[client_type] = websocket
    await func_dict[client_type](websocket)
    
    # Close socket
    state[client_type] = False
    websocket_dict.pop(client_type)
    await close_client(websocket, 'Finalized by server')

In [ ]:
uvicorn.run(app, host='0.0.0.0', port=8000)

In [5]:
from starlette.websockets import WebSocket

In [37]:
WebSocket.sen